# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = "..//Output_Data//city.csv"
cities = pd.read_csv(df)
cities.head()


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Albany,94,US,1659388843,49,42.6001,-73.9662,86.58,5.39
1,1,Jiayuguan City,100,CN,1659388869,63,39.8167,98.3000,65.35,7.65
2,2,Vaini,40,TO,1659388869,89,-21.2000,-175.2000,78.96,0.00
3,3,Alugan,100,PH,1659388870,91,12.2188,125.4808,76.75,3.76
4,4,Port Pirie,89,AU,1659388870,55,-33.1833,138.0167,48.90,10.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations
locations = cities[["Lat", "Lng"]]
weight = cities["Humidity"]
figure_layout = {
    'width': '300px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

#Create and add heat layer 
heat=gmaps.heatmap_layer(locations, weights=weight,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat)
#Display figure
fig 



Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#A max temperature lower than 80 degrees but higher than 70.
temp_file=cities.loc[(cities['Max Temp']>=70)&(cities['Max Temp']<=80)]
#Wind speed less than 10 mph.
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
#Zero cloudiness.
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
71,71,Shahr-e Bābak,0,IR,1659388890,52,30.1165,55.1186,75.72,1.36
95,95,Hyères,0,FR,1659388898,67,43.1167,6.1167,79.45,8.05
172,172,Tūkrah,0,LY,1659388923,74,32.5341,20.5791,79.95,9.33
173,173,Binzhou,0,CN,1659388923,83,37.3667,118.0167,75.83,5.03
198,198,Tunceli,0,TR,1659388932,26,39.1083,39.5471,77.32,6.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:


hotel_df = ideal_weather

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
#to continue on fails    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#a separator for my old eyes to see each result        
    print("------------")



Retrieving Results for Index 71: Shahr-e Bābak.
Closest hotel is سلاطین مس و نقره.
------------
Retrieving Results for Index 95: Hyères.


C:\Users\jess\AppData\Local\Temp\ipykernel_19332\3407661820.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


Closest hotel is ibis budget Hyères Centre-Ville.
------------
Retrieving Results for Index 172: Tūkrah.
Closest hotel is Lagrota restaurant.
------------
Retrieving Results for Index 173: Binzhou.
Closest hotel is Hampton By Hilton Binzhou.
------------
Retrieving Results for Index 198: Tunceli.
Closest hotel is Grand Hotel Şaroğlu.
------------
Retrieving Results for Index 257: Gairo.
Closest hotel is La Nuova Luna.
------------
Retrieving Results for Index 329: Vila Velha.
Closest hotel is Hotel Vitória Palace.
------------
Retrieving Results for Index 347: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 365: Dingzhou.
Closest hotel is 宜家时尚宾馆.
------------
Retrieving Results for Index 370: Neyshabur.
Closest hotel is Amiran Hotel & Commercial Complex.
------------
Retrieving Results for Index 381: Wanxian.
Closest hotel is 7Days Inn Chongqing Shiqiaopu.
------------
Retrieving Results for Index 441: Qiongshan.
Closest hotel is Sun City Restaur

In [6]:
hotel_df


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
71,71,Shahr-e Bābak,0,IR,1659388890,52,30.1165,55.1186,75.72,1.36,سلاطین مس و نقره
95,95,Hyères,0,FR,1659388898,67,43.1167,6.1167,79.45,8.05,ibis budget Hyères Centre-Ville
172,172,Tūkrah,0,LY,1659388923,74,32.5341,20.5791,79.95,9.33,Lagrota restaurant
173,173,Binzhou,0,CN,1659388923,83,37.3667,118.0167,75.83,5.03,Hampton By Hilton Binzhou
198,198,Tunceli,0,TR,1659388932,26,39.1083,39.5471,77.32,6.15,Grand Hotel Şaroğlu
257,257,Gairo,0,IT,1659388951,59,39.8497,9.5047,71.17,4.99,La Nuova Luna
329,329,Vila Velha,0,BR,1659388975,86,-20.3297,-40.2925,75.15,9.22,Hotel Vitória Palace
347,347,Abu Samrah,0,SY,1659388980,56,35.3029,37.1841,79.45,8.84,NaN
365,365,Dingzhou,0,CN,1659388986,84,38.5131,114.9956,78.91,2.13,宜家时尚宾馆
370,370,Neyshabur,0,IR,1659388988,40,36.2133,58.7958,75.81,2.59,Amiran Hotel & Commercial Complex


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…